In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
africa = pd.read_csv('../data/africa.csv')
asia = pd.read_csv('../data/asia.csv')
cen_am = pd.read_csv('../data/central_america.csv')
europe = pd.read_csv('../data/europe.csv')
mena = pd.read_csv('../data/mena.csv')
nor_am = pd.read_csv('../data/north_america.csv')
oceania = pd.read_csv('../data/oceania.csv')
sou_am = pd.read_csv('../data/south_america.csv')
world = pd.read_csv('../data/global.csv')

In [3]:
print(f'Africa: {africa.shape}')
print(f'Asia : {asia.shape}')
print(f'Central America : {cen_am.shape}')
print(f'Europe : {europe.shape}')
print(f'Middle East : {mena.shape}')
print(f'North America : {nor_am.shape}')
print(f'Oceania : {oceania.shape}')
print(f'South America : {sou_am.shape}')
print(f'Worldwide : {world.shape}')

Africa: (3184, 33)
Asia : (3096, 33)
Central America : (450, 33)
Europe : (4989, 33)
Middle East : (1257, 33)
North America : (525, 33)
Oceania : (38, 33)
South America : (1654, 33)
Worldwide : (15193, 33)
